# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Олег Черемисин`

Группа: `MADE-ML-32`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [ ]:
import folium

while(True):
    
    lat = np.random.rand() * 180 - 90
    lon = np.random.rand() * 360 - 180
    
    url = 'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat='+str(lat)+'&lon='+str(lon)+'&accept-language=en'

    req = requests.get(url).json()
    try:
        if req['address']['country_code'] == 'af':
            print('Found ', lat, lon)
            break
    except:
        pass


fmap = folium.Map([lat, lon])

folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='orange'),
        popup='point'
    ).add_to(fmap)
    
fmap

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [37]:
import json
import requests
from tqdm import tqdm

import folium
import pyproj
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

from openrouteservice import client

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
url = 'https://data.cityofnewyork.us/api/views/a3nt-yts4/rows.json?accessType=DOWNLOAD'

def CreateBufferPolygon(point_in, resolution=10, radius=10):    

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632') # WGS84 UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 10 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs
    

In [ ]:
-73.968285, 40.785091

-73.98806, 40.70306

In [57]:
# Set up the fundamentals
api_key = '5b3ce3597851110001cf62483414bd8dcf43473e9b60ea40e4aa0533' # Individual api key
clnt = client.Client(key=api_key) # Create client with api key
rostock_json = requests.get(url).json() # Get data as JSON

map_params = {'tiles':'Stamen Toner',
              'location':([40.769361, -73.977655]),
              'zoom_start': 12}
map1 = folium.Map(**map_params)

# Populate a construction site buffer polygon list
sites_poly = []

for item in tqdm(rostock_json['data']):
    coords = item[8]
    coords = coords.replace('POINT (','').replace(')','')
    
    # 1992 школ не влазили в фолиум, уменшим областью чуть шире заданных начала и конца пути
    site_coords = [float(x) for x in coords.split(' ')]

    if site_coords[0] < -73.96 and site_coords[0] > -73.99 and site_coords[1] > 40.70 and site_coords[0] < 40.79:

        folium.features.Marker(list(reversed(site_coords)),
                                   popup='School<br>{0}'.format(site_coords)).add_to(map1)

        # Create buffer polygons around construction sites with 10 m radius and low resolution
        site_poly_coords = CreateBufferPolygon(site_coords,
                                                   resolution=2, # low resolution to keep polygons lean
                                                   radius=10)
        sites_poly.append(site_poly_coords)

        site_poly_coords = [(y,x) for x,y in site_poly_coords] # Reverse coords for folium/Leaflet
        folium.vector_layers.Polygon(locations=site_poly_coords,
                                          color='#ffd699',
                                          fill_color='#ffd699',
                                          fill_opacity=0.2,
                                          weight=3).add_to(map1)
    
map1

100%|██████████| 1992/1992 [02:25<00:00, 13.72it/s]


In [61]:
# GeoJSON style function
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=0.5)

# Create new map to start from scratch
map_params.update({'location': ([40.769361, -73.977655]),
                   'zoom_start': 13})
map2 = folium.Map(**map_params)

# Request normal route between appropriate locations without construction sites
request_params = {'coordinates': [[-73.977655, 40.769361],
                                 [-73.98806, 40.70306]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}
route_normal = clnt.directions(**request_params)
folium.features.GeoJson(data=route_normal,
                        name='Route without Schools',
                        style_function=style_function('#FF0000'),
                        overlay=True).add_to(map2)

# Buffer route with 0.009 degrees (really, just too lazy to project again...)
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.009)
folium.features.GeoJson(data=geometry.mapping(route_buffer),
                        name='Route Buffer',
                        style_function=style_function('#FFFF00'),
                        overlay=True).add_to(map2)

# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map2)
        sites_buffer_poly.append(poly)

map2